In [12]:
import pandas as pd
import jieba
from collections import Counter
from tqdm import tqdm_notebook
import re
from gensim.models import Word2Vec # 建立word2vec模型
import pyltp
from pyltp import SentenceSplitter
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import NamedEntityRecognizer
from pyltp import Parser
from pyltp import SementicRoleLabeller
import os

In [2]:
def cut(string): return list(jieba.cut(string))

In [3]:
def token(string):
    return re.findall(r'[\w\d]+', string)

In [4]:
content = pd.read_csv('sqlResult_1558435.csv',encoding='gb18030')

In [5]:
content  = content.fillna('') # 去空值

In [6]:
all_content = ' '.join(content['content'].tolist())

In [7]:
all_content = ''.join(token(all_content))

In [8]:
all_cut_words = cut(all_content)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 0.766 seconds.
Prefix dict has been built successfully.


In [9]:
mini_all_cut_words = all_cut_words[:10000]
from gensim.models.word2vec import LineSentence
with open('all_cut_words.txt', 'w', encoding='utf-8') as f:
    f.write(' '.join(all_cut_words))
news_word2vec = Word2Vec(LineSentence('all_cut_words.txt'), size=35, workers=8)
keywords = news_word2vec.most_similar('说', topn=20)
key_word_list = [i[0] for i in keywords]
key_word_list.append('说')

D:\ProgramData\Anaconda3\Lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
D:\ProgramData\Anaconda3\Lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [13]:
LTP_DATA_DIR = r'F:\study\NLP\ltp_data_v3.4.0'
cws_model_path = os.path.join(LTP_DATA_DIR,  'cws.model')
pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')
ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`
srl_model_path = os.path.join(LTP_DATA_DIR, 'srl')  # 语义角色标注模型目录路径，模型目录为`srl`。注意该模型路径是一个目录，而不是一个文件。

In [14]:
def sentence_splitter(sentence):
#     sents = SentenceSplitter.split(sentence)
    sents = sentence
#     print('\n'.join(sents))
    sents_list = list(sents)
    return sents_list

In [15]:
#分词   
def segmentor(sentence):
    segmentor =  Segmentor()
    segmentor.load(cws_model_path)#加载模型
    words = segmentor.segment(sentence) #分词
    #默认可以这样输出
#     print ('\t'.join(words) + '\n')
    #可以转化成List输出
    word_list = list(words)
    segmentor.release()#释放模型
    return word_list

In [16]:
#词性标注
def posttagger(words):
    postagger = Postagger()
    postagger.load(pos_model_path)
    posttags=postagger.postag(words)  #词性标注
    postags = list(posttags)
    postagger.release() #释放模型
    #print type(postags)
    return postags

In [17]:
#依存句法分析
def par(words,postags):
    parser = Parser() # 初始化实例
    parser.load(par_model_path)  # 加载模型
    arcs = parser.parse(words, postags)  # 句法分析
#     for word,ntag in zip(words,netags):
#         print(word+'/'+ ntag)
#     print("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs)
    parser.release()  # 释放模型
    return arcs

In [18]:
# 语义角色标注
def sem(words, postags,arcs):
    labeller = SementicRoleLabeller() # 初始化实例
    labeller.load(srl_model_path)  # 加载模型
    roles = labeller.label(words, postags,arcs)  # 语义角色标注
    print('ok')
    print(roles)
    print ('*'*8)
    
#     for role in roles:
#         print (role.index, "".join(
#         ["%s:(%d,%d)" % (arg.name, arg.range.start, arg.range.end) for arg in role.arguments]))
    labeller.release()  # 释放模型    
    return roles

In [21]:
#命名实体识别
def ner(words,postags):
    recognizer = NamedEntityRecognizer()
    recognizer.load(ner_model_path) #加载模型
    netags = recognizer.recognize(words,postags) #命名实体识别
#     for word,ntag in zip(words,netags):
#         print(word+'/'+ ntag)
    recognizer.release()   #释放模型
    nerttags = list(netags)
    return nerttags

In [29]:
test_news ="""刘德华坦言2019年可以再开一个世界巡回演唱会；元芳说想吃饺子；狄仁杰表示他也喜欢吃；黄飞鸿表示他今年还要拍一部电影；"""

In [30]:
sents = SentenceSplitter.split(test_news)  # 分句
for sent in sents:
    words=segmentor(sent)
    tags = posttagger(words)
    nertags = ner(words,tags)
    arcs = par(words, tags)
    arcstags = list(arcs)
    ans_arcs = ["%d:%s" % (arc.head, arc.relation) for arc in arcstags]
    for word,nertag in zip(words,nertags):
        if 'Nh' in nertag:
            print(word)
    for word,ans_arc in zip(words,ans_arcs):
        if(ans_arc == '0:HED') & (word in key_word_list):
            index_word = words.index(word)+1
            print("".join(arg for arg in words[index_word:-1]))
    print('\n'+ "*"*8)

刘德华
2019年可以再开一个世界巡回演唱会

********
元芳
想吃饺子

********
狄仁杰
他也喜欢吃

********
黄飞鸿
他今年还要拍一部电影

********
